<a href="https://colab.research.google.com/github/itchyfeet-patient/Beautiful-Exploration/blob/master/Exploration_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트: 멋진 작사가 만들기 🎹

## 라이브러리 버전을 확인해 봅니다

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.8.2


## Step 1. 데이터 다운로드

aiffel 클라우드 주피터에 있는 파일을 받아왔습니다!

## Step 2. 데이터 읽어오기

glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [ ]:
import glob
import os

txt_file_path = '/content/drive/MyDrive/dataset/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


## Step 3. 데이터 정제


앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [ ]:
corpus =[]

for sentence in raw_corpus:
   # if len(sentence.split(' ')) > 15: continue
    sentence = '<start> ' + sentence + ' <end>'
    corpus.append(sentence) # 담기
    
corpus[:10]

['<start> Looking for some education <end>',
 '<start> Made my way into the night <end>',
 '<start> All that bullshit conversation <end>',
 "<start> Baby, can't you read the signs? I won't bore you with the details, baby <end>",
 "<start> I don't even wanna waste your time <end>",
 "<start> Let's just say that maybe <end>",
 '<start> You could help me ease my mind <end>',
 "<start> I ain't Mr. Right But if you're looking for fast love <end>",
 "<start> If that's love in your eyes <end>",
 "<start> It's more than enough <end>"]

## Step 4. 평가 데이터셋 분리

훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!



In [ ]:
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words= 12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 292  22 ...   0   0   0]
 [  2 214  10 ...   0   0   0]
 [  2  21  14 ...   0   0   0]
 ...
 [  2  92   4 ...   0   0   0]
 [  2 116   9 ...   0   0   0]
 [  2  60   4 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fa9b70f4710>


In [ ]:
print(tensor[:3, :10])

[[   2  292   22   87 6868    3    0    0    0    0]
 [   2  214   10   79  215    4  127    3    0    0]
 [   2   21   14 1127 2769    3    0    0    0    0]]


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : i
6 : you
7 : and
8 : to
9 : a
10 : my


In [ ]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2  292   22   87 6868    3    0    0    0    0    0    0    0    0]
[ 292   22   87 6868    3    0    0    0    0    0    0    0    0    0]


In [ ]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=2022)

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

## Step 5. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!

잘 설계한 모델을 학습하려면, model.fit() 함수를 사용해야 합니다. model.fit() 함수에는 다양한 인자를 넣어주어야 하는데, 가장 기본적인 인자로는 데이터셋과 epochs가 있습니다. '5. 실습 (2) 인공지능 학습시키기'에서의 예시와 같이 말이죠.

model.fit(dataset, epochs=30)  
하지만 model.fit() 함수의 epochs를 아무리 크게 넣는다 해도 val_loss 값은 2.2 아래로 떨어지지 않습니다. 이럴 경우는 batch size를 변경하는 것과 같이 model.fit() 함수에 다양한 인자를 넣어주면 해결될 수도 있습니다. 자세한 내용은 https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit 를 참고하세요!

Loss는 아래 제시된 Loss 함수를 그대로 사용하세요!

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        # 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔줌
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, dropout = 0.3)
        
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 2048
# 워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기
hidden_size = 2048
# 모델에 얼마나 많은 일꾼을 둘 것인가?

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

dropout을 해주면 정규화가 된다고 해서 첫번째 rnn층에 0.3 비율로 해줬습니다.  
embedding_size와 hidden_size를 2048로 바꿔봤습니다.

In [ ]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-4.34777816e-04,  6.19832077e-04, -2.90790540e-05, ...,
         -2.02905823e-04, -3.64403852e-04,  6.13549957e-04],
        [-6.31678035e-04,  9.67872096e-04, -5.30990423e-04, ...,
         -7.63465214e-05, -1.37594249e-03,  8.99100385e-04],
        [-1.01788272e-03,  1.32904842e-03, -1.09078363e-03, ...,
          8.31643352e-04, -1.35210564e-03,  1.19861064e-03],
        ...,
        [ 2.31017405e-03, -1.30087649e-03, -2.80662160e-03, ...,
         -1.12886206e-04,  1.12376455e-03,  1.53059140e-03],
        [ 2.85678380e-03, -1.16870564e-03, -2.67065014e-03, ...,
         -2.57954176e-04,  9.69151559e-04,  2.14233319e-03],
        [ 3.17313615e-03, -1.03911071e-03, -2.26436532e-03, ...,
         -3.76478478e-04,  8.21163936e-04,  2.94275302e-03]],

       [[-4.34777816e-04,  6.19832077e-04, -2.90790540e-05, ...,
         -2.02905823e-04, -3.64403852e-04,  6.13549957e-04],
        [-2.77981017e-04,  8.82921217e-04, -1

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24578048  
                                                                 
 lstm (LSTM)                 multiple                  33562624  
                                                                 
 lstm_1 (LSTM)               multiple                  33562624  
                                                                 
 dense (Dense)               multiple                  24590049  
                                                                 
Total params: 116,293,345
Trainable params: 116,293,345
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Loss
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

model.fit(dataset, epochs=10, validation_data = val_dataset, verbose = 1)

Epoch 1/10
584/584 [==============================] - 198s 331ms/step - loss: 3.1398 - accuracy: 0.5289 - val_loss: 2.7618 - val_accuracy: 0.5549
Epoch 2/10
584/584 [==============================] - 193s 329ms/step - loss: 2.5731 - accuracy: 0.5686 - val_loss: 2.4836 - val_accuracy: 0.5812
Epoch 3/10
584/584 [==============================] - 193s 329ms/step - loss: 2.2127 - accuracy: 0.6035 - val_loss: 2.3107 - val_accuracy: 0.6050
Epoch 4/10
584/584 [==============================] - 193s 330ms/step - loss: 1.8784 - accuracy: 0.6447 - val_loss: 2.1931 - val_accuracy: 0.6280
Epoch 5/10
584/584 [==============================] - 193s 329ms/step - loss: 1.5785 - accuracy: 0.6921 - val_loss: 2.1196 - val_accuracy: 0.6479
Epoch 6/10
584/584 [==============================] - 192s 329ms/step - loss: 1.3323 - accuracy: 0.7371 - val_loss: 2.0886 - val_accuracy: 0.6633
Epoch 7/10
584/584 [==============================] - 193s 330ms/step - loss: 1.1460 - accuracy: 0.7754 - val_loss: 2.0882 -

val_loss 가 2.1604로 만족스럽습니다?`

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> life", max_len=20)

'<start> life is worth living again <end> '

# ✍ 회고

1. 처음에 수동적으로 따라하다 보니 validation data를 split 해놓고도 학습안시킴. 이거 문제있다.... 능동적으로 살자 정신체리 🍒.
2. **embedding size와 hidden size를** 높였더니 val_loss가 급격히 낮아졌다. 처음 생각으로는 **hidden size**가 일꾼들의 개수라고 해서, 많으면 배가 산으로 간다기에 좀 적게 잡았더니 만족할 만한 수치가 나오지 않았다. 오히려 지금 데이터 수가 많아서 2048이 적은 수치가 아니었던 것일까? 의문의 연속이다.
3. 데이터 수가 많아졌다 보니 좀 과적합? 될수도 있겠다는 생각이 들어서 어제 카훗에 잠깐 나온 **dropout을 사용해봤다**. RNN 구글링 하다 보니 RNN 층에 옵션으로 dropout 비율을 지정해주는 경우가 있어서. **0.3**으로 지정해 줬더니 조금 ? 빨라진 것 같기도 하고 val_loss가 감소된 것 같기도 하고..   
    그런데 결과의 정확도를 위해서는 dropout을 지양한다고 한다. 사실 그렇지. 가지치기를 하는데 완전 정확하진 않겠지?
4. **학습시간이 길다**보니 다양한 시도를 해 보기도 어려웠고 매번 어떤 파라미터를 고쳐서 이 결과가 나온건지 모호했다. (5번에서 느낀 것 처럼)  
    그래서 꼼수로 epoch를 조금 돌려봐서 가망있는(?) val_loss 수치가 나오면 epoch를 늘려서 결과를 뽑았다. 크크
5. tensorflow가 참 간편하다. tensorflow의 함수로 tokenize도 하고 사전 인덱스값을 워드 벡터로도 바꿔주고.. 쵝오
6. 처음에 `if len(sentence.split(' ')) > 15: continue` 로 토큰의 개수를 조절했는데, **tensor의 구조가 15를 넘는것**에 뭔가 이상하고 느꼈고 토큰의 개수를 조절하는 것이므로 tokenize 할 때 maxlen을 조절하는 것이 맞다고 생각해서 수정했다.
7. 문장을 만든 걸 보니 다 기존에 있던 가사의 문장이 통으로 자주 나왔는데 이러면 **표절시비**에 휘말리지 않을까? 하는 생각이 듬  
    그래서 단어의 tokenize가 잘 안된게 아닌지 확인해봤는데 그건 상관없었고...  
    예측된 값 중 확률이 가장 높은 확률이 공교롭게도 그것이었던 것이었던 것이었을까..? 
8. 학습한 모델로 처음 작사 시켰을 때 아는 노래가 나왔다... Eminem의 Love the way you lie... 엄청 인기있었던 라떼노랜데 요즘애들 알까? 니 덕분에 오랜만에 들었어 고마워 인공지능친구야! 🤖
9. 아무리 사랑타령하는 노래가 많다지만 i love 로 시작하는 문장은 식상하다고 생각해서 life로 시작하는 문장을 만들어보라고 시켰더니 **'life is worth living again'** 라는 저스틴비버의 노래 제목이 나왔다. 인생은 다시 살아볼만 하지!

## 🎯 평가 루브릭
| **평가문항** | **상세기준** | **학습결과** |
|---|---|:---:|
| 1. 가사 텍스트 생성 모델이 정상적으로 동작하는가? | 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가? | O |
| 2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가? | 특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가? | O |
| 3. 텍스트 생성모델이 안정적으로 학습되었는가? | 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가? | Validation loss : 2.1604 |

### 참고자료
Dropout : https://deepestdocs.readthedocs.io/en/latest/004_deep_learning_part_2/0042/  
RNN : https://davinci-ai.tistory.com/30